In [1]:
import os
import torch
import numpy as np
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [27]:
from models.model import build_model
model = build_model(tokenizer.vocab_size,512,2048,64,12,'cpu')
weight = torch.load('iter_110000.pth',map_location=torch.device('cpu'))['model_state_dict']

In [28]:
model_weight = {}
for key,val in weight.items():
    if key.startswith('module.'):
        model_weight[key[7:]] = val
    else:
        print(key)
model.load_state_dict(model_weight)

<All keys matched successfully>

In [29]:
def inference(text,model,tokenizer,max=64,mask='[MASK]'):
     input  = tokenizer(text,max_length=max,padding='max_length',return_tensors='pt')
     tokens = tokenizer.convert_ids_to_tokens(input['input_ids'].numpy().squeeze())
     idx = tokens.index(mask)
     output = model(input['input_ids'],input['token_type_ids']).squeeze()
     masked_input = output[idx].detach().numpy()
     predicted_vocab = np.argmax(masked_input)
     predicted_vocab = tokenizer.convert_ids_to_tokens([predicted_vocab])
     return predicted_vocab


model= model.eval()

In [30]:
text = 'I want to [MASK] coffee.'
inference(text,model,tokenizer)

tensor(0.3327)
tensor(0.6232)
tensor(0.4975)
tensor(0.4493)
tensor(0.3444)
tensor(0.6129)
tensor(0.6101)
tensor(0.3525)
tensor(0.5657)
tensor(0.6557)
tensor(0.6757)
tensor(0.2471)


['make']

In [31]:
text = 'I am hungry so I want to [MASK] steak.'
inference(text,model,tokenizer)

tensor(0.3345)
tensor(0.5313)
tensor(0.5038)
tensor(0.4145)
tensor(0.4530)
tensor(0.5633)
tensor(0.6120)
tensor(0.4372)
tensor(0.6078)
tensor(0.6676)
tensor(0.5590)
tensor(0.2155)


['eat']

In [33]:
text = 'This function [MASK] differentiable, so gradients will flow back from the result of this operation.'
inference(text,model,tokenizer)

tensor(0.3763)
tensor(0.5854)
tensor(0.2789)
tensor(0.4430)
tensor(0.4672)
tensor(0.6326)
tensor(0.6319)
tensor(0.3933)
tensor(0.4384)
tensor(0.7933)
tensor(0.6049)
tensor(0.2452)


['is']

In [34]:
text = 'This function is differentiable, so gradients [MASK] flow back from the result of this operation.'
inference(text,model,tokenizer)

tensor(0.3355)
tensor(0.6006)
tensor(0.3058)
tensor(0.4079)
tensor(0.4934)
tensor(0.5693)
tensor(0.6565)
tensor(0.3596)
tensor(0.4127)
tensor(0.7644)
tensor(0.6440)
tensor(0.2778)


['will']